In [ ]:
import pandas as pd

In [ ]:
filepath = './buildings/990/rssi_2018_02.csv'

In [ ]:
df = pd.read_csv(filepath)

In [ ]:
df.head(50)

In [ ]:
# df_ordenado = df.sort_values(by=['time', 'date'], ascending=[True, True])

In [ ]:
df_1 = df.sort_values(by=['mac_cliente','time','date'])

In [ ]:
df_1.head(50)

In [ ]:
df_1['mac_ap'].unique()
# df_1.columns

Imputer: Para cada cliente en cada tiempo, quiero que tenga info de todos los APs en cada banda.

Obs: como esta ahora el imputer crea timestamps nuevos para los clientes si no existian en ese momento. no deberia hacer esto, sino solo crear las filas para los timestamps en los que existe ese cliente. como arreglo rapido hice que se eliminaran al final, pero habria que revisar que funcione bien.

In [ ]:
def impute_rssi_all_aps(df: pd.DataFrame, rssi_fill: float = -101) -> pd.DataFrame:
    """
    Para cada cliente y cada instante de tiempo, garantiza que existan
    todas las combinaciones (AP, banda), imputando los RSSI faltantes.

    Parámetros
    ----------
    df : pd.DataFrame
        Debe contener las columnas:
        ['mac_cliente', 'mac_ap', 'banda', 'rssi', 'date', 'time']
    rssi_fill : float, default -101
        Valor para imputar RSSI faltantes.

    Returns
    -------
    pd.DataFrame
        DataFrame con todas las combinaciones completas e imputadas.
    """

    df = df.copy()

    # 1) Crear timestamp unificado
    if 'timestamp' not in df.columns:
        df['timestamp'] = pd.to_datetime(df['date'] + ' ' + df['time'])

    # 2) Valores unicos para obtener todas las combinaciones
    clientes = df['mac_cliente'].unique()
    timestamps = df['timestamp'].unique()
    aps = df['mac_ap'].unique()
    bandas = df['banda'].unique()
    antenas = df['antena'].unique()

    # 3) Índice completo
    full_index = pd.MultiIndex.from_product(
        [clientes, timestamps, aps, bandas, antenas],
        names=['mac_cliente', 'timestamp', 'mac_ap', 'banda', 'antena']
    )

    # 4) Reindexar
    df_full = (
        df
        .set_index(['mac_cliente', 'timestamp', 'mac_ap', 'banda', 'antena'])
        .reindex(full_index)
    )

    # 5) Imputar RSSI
    df_full['rssi'] = df_full['rssi'].fillna(rssi_fill)

    # 6) Volver a columnas normales
    df_full = df_full.reset_index()

    # 7) Reconstruir date y time desde timestamp
    df_full['date'] = df_full['timestamp'].dt.strftime('%Y-%m-%d')
    df_full['time'] = df_full['timestamp'].dt.strftime('%H:%M:%S')

    # 8) Eliminar timestamps "falsos":
    # aquellos donde TODOS los RSSI del cliente son rssi_fill
    mask_valid = (
        df_full
        .groupby(['mac_cliente', 'timestamp'])['rssi']
        .transform(lambda x: (x != rssi_fill).any())
    )

    df_full = df_full[mask_valid]

    # 9) Borrar timestamp
    df_full.drop(columns=['timestamp'], inplace=True)

    return df_full


In [ ]:
df_imputed = impute_rssi_all_aps(df)

df_imputed.head(20)

In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

In [ ]:
filepath = './buildings/990/rssi_2018_11.csv'
df = pd.read_csv(filepath)

In [ ]:
df=df_imputed

In [ ]:
# --- CONFIGURACIÓN ---
NOMBRE_ARCHIVO_SALIDA = 'simulacion_bloques_reales.csv'
TOTAL_TIMEFRAMES = 100
PROBABILIDAD_APARICION = 0.8

# --- 1. CARGA Y POOL ---
df['timestamp_real'] = pd.to_datetime(df['date'] + ' ' + df['time'])
# Ordenamos cronológicamente para respetar la historia del usuario
df_ordenado = df.sort_values(by=['mac_cliente', 'timestamp_real'])

pool_usuarios = {}

for mac, df_user in df_ordenado.groupby('mac_cliente'):
    # 1. Detectamos los tiempos ÚNICOS (Los bloques verdaderos)
    tiempos_unicos = df_user['timestamp_real'].unique()
    
    lista_bloques = []
    for t_real in tiempos_unicos:
        # 2. Agarramos TODAS las filas de ese instante (el senseo completo)
        bloque = df_user[df_user['timestamp_real'] == t_real].copy()
        lista_bloques.append(bloque)
    
    # El pool es una lista de dataframes: [Bloque_11:05, Bloque_11:06, Bloque_11:20...]
    pool_usuarios[mac] = lista_bloques

# --- 2. SIMULACIÓN ---
resultados = []
usuarios_ocupados = [] 

for t in range(1, TOTAL_TIMEFRAMES + 1):
    
    # A. Liberar ocupados
    usuarios_ocupados = [u for u in usuarios_ocupados if u['libre_en_t'] > t]
    macs_en_uso = [u['mac'] for u in usuarios_ocupados]

    # B. Candidatos (Tienen bloques en el pool y no están usándose)
    candidatos = [mac for mac, bloques in pool_usuarios.items() if len(bloques) > 0 and mac not in macs_en_uso]

    # C. Sorteo
    if candidatos and random.random() < PROBABILIDAD_APARICION:
        
        mac_elegida = random.choice(candidatos)
        bloques_disponibles = pool_usuarios[mac_elegida]
        n_disponibles = len(bloques_disponibles)
        
        # --- EL SORTEO DE DURACIÓN ---
        # Sorteamos entre 1 y la cantidad de bloques reales que le quedan
        duracion = random.randint(1, n_disponibles)
        
        # Consumimos la secuencia exacta de bloques
        bloques_a_usar = bloques_disponibles[:duracion]
        pool_usuarios[mac_elegida] = bloques_disponibles[duracion:] # Sacamos del pool
        
        # Asignamos Timeframes consecutivos
        for i, df_bloque in enumerate(bloques_a_usar):
            df_bloque = df_bloque.copy()
            
            # La magia: El bloque real 1 va a T, el bloque real 2 va a T+1...
            df_bloque['timeframe'] = t + i  
            
            resultados.append(df_bloque)
            
        usuarios_ocupados.append({
            'mac': mac_elegida,
            'libre_en_t': t + duracion
        })

# --- 3. EXPORTAR ---
if resultados:
    df_final = pd.concat(resultados).reset_index(drop=True)
    df_final = df_final.sort_values(['timeframe', 'mac_cliente'])
    
    # Guardamos
    df_final.to_csv(NOMBRE_ARCHIVO_SALIDA, index=False)
    
    print("\n--- EJEMPLO DE SECUENCIA ---")
    # Verificamos si logramos secuencias > 1
    ejemplo = df_final[['timeframe', 'mac_cliente', 'time', 'date']].head(20)
    print(ejemplo)
else:
    print("No se generaron datos.")

In [ ]:
# 1. Cargar el resultado de tu simulación
df = pd.read_csv('simulacion_bloques_reales.csv')

# 2. Crear la matriz (Pivot)
# Contamos cuántas filas hay por (usuario, timeframe)
pivot_df = df.pivot_table(index='mac_cliente', columns='timeframe', aggfunc='size', fill_value=0)

# 3. Graficar
fig, ax = plt.subplots(figsize=(15, 8))

# Dibujar la matriz (1 si está ocupado, 0 si no)
# Usamos un mapa de colores 'Blues' o 'GnBu'
data_matrix = (pivot_df > 0).astype(int)
ax.imshow(data_matrix, cmap='Blues', aspect='auto', interpolation='nearest')

# Configurar la grilla visual
ax.set_xticks(np.arange(-.5, len(pivot_df.columns), 1), minor=True)
ax.set_yticks(np.arange(-.5, len(pivot_df.index), 1), minor=True)
ax.grid(which='minor', color='lightgray', linestyle='-', linewidth=0.5)

# Etiquetas de los ejes
ax.set_xticks(np.arange(len(pivot_df.columns)))
ax.set_xticklabels([f"t{int(c)}" for c in pivot_df.columns], rotation=45)
ax.set_yticks(np.arange(len(pivot_df.index)))
ax.set_yticklabels([f"u_{i+1}" for i in range(len(pivot_df.index))])

# Agregar el texto txu1, txu2... dentro de las celdas
for i in range(len(pivot_df.index)):
    for j in range(len(pivot_df.columns)):
        if pivot_df.iloc[i, j] > 0:
            ax.text(j, i, f"u{i+1}\nt{j+1}", ha="center", va="center", 
                    color="white", fontsize=8, fontweight='bold')

ax.set_title("Grilla de Simulación: Distribución de Usuarios en Timeframes", fontsize=16)
ax.set_xlabel("Línea de Tiempo (t)", fontsize=12)
ax.set_ylabel("Usuarios (u)", fontsize=12)

plt.tight_layout()
plt.savefig('mi_grilla_visual.svg')

ASIGNACIONES:

In [ ]:
# def asignar_ap_por_usuario(df: pd.DataFrame, umbral_5g: float = -75) -> pd.DataFrame:
#     """
#     Asigna un único AP por usuario y timeframe según criterio:
#     - En la primera aparición del usuario:
#         * elegir mejor AP en 5GHz si RSSI > umbral
#         * si no, elegir mejor AP en 2.4GHz
#     - En timeframes siguientes:
#         * mantener el AP elegido inicialmente
#     """

#     df = df.copy()
#     df = df.sort_values(['mac_cliente', 'timeframe'])

#     resultados = []

#     for mac, g in df.groupby('mac_cliente'):
#         # Timeframe inicial del usuario
#         t0 = g['timeframe'].min()
#         g_t0 = g[g['timeframe'] == t0]

#         # 1) Intentar 5 GHz
#         g_5g = g_t0[g_t0['banda'] == 1]
#         ap_elegido = None

#         if not g_5g.empty:
#             fila_5g = g_5g.loc[g_5g['rssi'].idxmax()]
#             if fila_5g['rssi'] > umbral_5g:
#                 ap_elegido = fila_5g['mac_ap']

#         # 2)si no pude 5G, busco en 2.4 GHz
#         if ap_elegido is None:
#             g_24 = g_t0[g_t0['banda'] == 0]
#             if not g_24.empty:
#                 ap_elegido = g_24.loc[g_24['rssi'].idxmax()]['mac_ap']
#             else:
#                 # Caso extremo: no hay nada válido
#                 continue

#         # 3️3) Filtrar TODA la secuencia del usuario a ese AP
#         g_filtrado = g[g['mac_ap'] == ap_elegido].copy()
#         g_filtrado['ap_asignado'] = ap_elegido

#         resultados.append(g_filtrado)

#     return pd.concat(resultados, ignore_index=True)


In [ ]:
def asignar_ap_por_usuario(
    df: pd.DataFrame,
    umbral_5g: float = -75,
    umbral_handover: float = -85
) -> pd.DataFrame:
    """
    Asigna un AP por usuario y timeframe.
    Re-evalúa la elección si el RSSI del AP actual cae por debajo de umbral_handover.
    """

    df = df.copy()
    df = df.sort_values(['mac_cliente', 'timeframe'])

    resultados = []

    for mac, g in df.groupby('mac_cliente'):
        g = g.sort_values('timeframe')
        ap_actual = None

        for t, g_t in g.groupby('timeframe'):
            # --- Primera vez o handover ---
            if ap_actual is None:
                reelegir = True
            else:
                fila_actual = g_t[g_t['mac_ap'] == ap_actual]
                reelegir = fila_actual['rssi'].iloc[0] < umbral_handover

            if reelegir:
                # 1) Intentar 5 GHz
                g_5g = g_t[g_t['banda'] == 1]
                ap_nuevo = None

                if not g_5g.empty:
                    fila_5g = g_5g.loc[g_5g['rssi'].idxmax()]
                    if fila_5g['rssi'] > umbral_5g:
                        ap_nuevo = fila_5g['mac_ap']

                # 2) Fallback a 2.4 GHz
                if ap_nuevo is None:
                    g_24 = g_t[g_t['banda'] == 0]
                    if not g_24.empty:
                        ap_nuevo = g_24.loc[g_24['rssi'].idxmax()]['mac_ap']
                    else:
                        continue  # no hay nada usable

                ap_actual = ap_nuevo

            # Guardar SOLO la fila del AP asignado
            fila_final = g_t[g_t['mac_ap'] == ap_actual].copy()
            fila_final['ap_asignado'] = ap_actual

            resultados.append(fila_final)

    return pd.concat(resultados, ignore_index=True)


In [ ]:
df_asignado = asignar_ap_por_usuario(df)

# chequeo que la cantidad de datos este bien
print(len(df)) 
print(df['mac_ap'].nunique())
print(len(df)/df['mac_ap'].nunique())

print(len(df_asignado)) 

In [ ]:
# 1. Cargar el resultado de tu simulación
df = df_asignado

# 2. Crear la matriz (Pivot)
# Contamos cuántas filas hay por (usuario, timeframe)
pivot_df = df.pivot_table(index='mac_cliente', columns='timeframe', aggfunc='size', fill_value=0)

# 3. Graficar
fig, ax = plt.subplots(figsize=(15, 8))

# Dibujar la matriz (1 si está ocupado, 0 si no)
# Usamos un mapa de colores 'Blues' o 'GnBu'
data_matrix = (pivot_df > 0).astype(int)
ax.imshow(data_matrix, cmap='Blues', aspect='auto', interpolation='nearest')

# Configurar la grilla visual
ax.set_xticks(np.arange(-.5, len(pivot_df.columns), 1), minor=True)
ax.set_yticks(np.arange(-.5, len(pivot_df.index), 1), minor=True)
ax.grid(which='minor', color='lightgray', linestyle='-', linewidth=0.5)

# Etiquetas de los ejes
ax.set_xticks(np.arange(len(pivot_df.columns)))
ax.set_xticklabels([f"t{int(c)}" for c in pivot_df.columns], rotation=45)
ax.set_yticks(np.arange(len(pivot_df.index)))
ax.set_yticklabels([f"u_{i+1}" for i in range(len(pivot_df.index))])

# Agregar el texto txu1, txu2... dentro de las celdas
for i in range(len(pivot_df.index)):
    for j in range(len(pivot_df.columns)):
        if pivot_df.iloc[i, j] > 0:
            ax.text(j, i, f"u{i+1}\nt{j+1}", ha="center", va="center", 
                    color="white", fontsize=8, fontweight='bold')

ax.set_title("Grilla de Simulación: Distribución de Usuarios en Timeframes", fontsize=16)
ax.set_xlabel("Línea de Tiempo (t)", fontsize=12)
ax.set_ylabel("Usuarios (u)", fontsize=12)

plt.tight_layout()
plt.savefig('mi_grilla_visual.svg')

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
FILE_PATH = "../buildings/990/building_990_all_months.csv"

In [5]:
df = pd.read_csv(FILE_PATH)

In [6]:
df.describe()

,banda,antena,rssi,client_index,block_index
count,23135.000000,23135.000000,23135.000000,23135.000000,23135.000000
mean,0.253728,0.500022,-75.095742,107.507759,43.041625
std,0.435153,0.500011,19.065143,52.376649,53.081289
min,0.000000,0.000000,-100.000000,1.000000,1.000000
25%,0.000000,0.000000,-93.000000,78.000000,6.000000
50%,0.000000,1.000000,-79.000000,106.000000,19.000000
75%,1.000000,1.000000,-59.000000,137.000000,62.000000
max,1.000000,1.000000,-26.000000,203.000000,229.000000


In [12]:
df[df['rssi'] >= -40].describe()

,banda,antena,rssi,client_index,block_index
count,874.000000,874.000000,874.000000,874.000000,874.000000
mean,0.061785,0.475973,-36.970252,109.762014,52.053776
std,0.240902,0.499708,2.642768,44.226669,52.937648
min,0.000000,0.000000,-40.000000,5.000000,1.000000
25%,0.000000,0.000000,-39.000000,82.000000,11.000000
50%,0.000000,0.000000,-37.000000,106.000000,26.000000
75%,0.000000,1.000000,-36.000000,137.000000,89.000000
max,1.000000,1.000000,-26.000000,195.000000,226.000000
